In [1]:
import sys
sys.path.append('..')
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sfp_nsdsyn import *
import warnings
import itertools
import matplotlib as mpl

warnings.filterwarnings("ignore", category=RuntimeWarning)
pd.options.mode.chained_assignment = None
%load_ext autoreload
%autoreload 2

In [2]:
output_dir='/Volumes/server/Projects/sfp_nsd/derivatives'
results_dir = os.path.join(output_dir, 'sfp_model', 'results_2D')
fig_dir = os.path.join(output_dir, 'figures', 'sfp_model', 'results_2D')
lr = 0.0005
max_epoch = 30000
dset='nsdsyn'
roi_list = ['V1','V2','V3']
vsc='pRFsize'
nsd_subj_list = [utils.sub_number_to_string(sn, 'nsdsyn') for sn in np.arange(1,9)]
broderick_sn_list = [1, 6, 7, 45, 46, 62, 64, 81, 95, 114, 115, 121]
broderick_subj_list = [utils.sub_number_to_string(i, 'broderick') for i in broderick_sn_list]
ARGS_2D = ['sub','lr','eph','roi','dset']

# NSD 
## model

In [3]:
pt_file_list = []
dset = 'nsdsyn'
for subj, roi in itertools.product(nsd_subj_list, ['V1','V2','V3']):
    file_name = os.path.join(results_dir, dset, f'model-params_lr-{lr}_eph-{max_epoch}_dset-{dset}_sub-{subj}_roi-{roi}_vs-{vsc}.pt')
    pt_file_list.append(file_name)
nsd_params = model.load_all_models(pt_file_list, *ARGS_2D)

## NSD precision

In [4]:
nsd_precision_s =  pd.read_csv(os.path.join(output_dir, 'dataframes', dset, 'precision', f'precision-s_dset-{dset}_vs-pRFsize.csv'))
nsd_df = pd.merge(nsd_params, nsd_precision_s[['sub','vroinames','precision']], on=['sub','vroinames'])

# Broderick et al
## model

In [5]:
pt_file_list = []
dset = 'broderick'
tfunc_types=['corrected']
for subj, roi, tfunc in itertools.product(broderick_subj_list, ['V1'], tfunc_types):
    file_name = os.path.join(results_dir, dset, f'tfunc-{tfunc}_model_lr-{lr}_eph-{max_epoch}_dset-{dset}_sub-{subj}_roi-{roi}_vs-{vsc}.pt')
    pt_file_list.append(file_name)
broderick_params = model.load_all_models(pt_file_list, *ARGS_2D+['tfunc'])
broderick_params.replace({'tfunc': {'derick/tfunc corrected': 'corrected', 'derick/tfunc uncorrected': 'uncorrected'}}, inplace=True)

## broderick precision

In [8]:
broderick_precision_s =  pd.read_csv(os.path.join(output_dir, 'dataframes', dset, 'precision', f'precision-s_dset-{dset}_vs-pRFsize.csv'))
all_broderick_df = pd.merge(broderick_params, broderick_precision_s[['sub','vroinames','precision']], on=['sub','vroinames'])

In [9]:
broderick_df = all_broderick_df.query('tfunc == "corrected"')
broderick_df['dset_type'] = 'Broderick et al. V1'
nsd_df['dset_type'] = nsd_df['vroinames'].apply(lambda x: f'NSD {x}')
final_df = pd.concat((broderick_df, nsd_df), axis=0)

In [10]:
final_df['goal'] = final_df['vroinames'].apply(lambda x: 'Replication' if x == "V1" else 'Extension')
tmp = nsd_df.query('vroinames == "V1"')
tmp['goal'] = 'Extension'
final_df = pd.concat((tmp, final_df), axis=0)


In [11]:
final_df

,sigma,slope,intercept,p_1,p_2,p_3,p_4,A_1,A_2,sub,lr_rate,max_epoch,vroinames,dset,precision,dset_type,goal,tfunc
0,2.490021,0.139594,0.217478,0.022201,-0.021813,-0.274858,-0.012191,0.023888,-0.006918,subj01,0.0005,30000,V1,nsdsyn,1.342082,NSD V1,Extension,NaN
3,2.009928,0.124187,0.090198,0.101903,-0.053708,-0.001964,-0.054230,0.006940,0.006901,subj02,0.0005,30000,V1,nsdsyn,0.758509,NSD V1,Extension,NaN
6,2.529316,0.054894,0.109951,0.045723,0.009979,0.132809,-0.127411,0.005100,-0.009297,subj03,0.0005,30000,V1,nsdsyn,0.451902,NSD V1,Extension,NaN
9,1.979350,0.182145,0.122061,0.046359,-0.014748,0.117450,-0.143140,0.036360,-0.017871,subj04,0.0005,30000,V1,nsdsyn,1.112438,NSD V1,Extension,NaN
12,2.611645,0.110086,0.135820,-0.044682,-0.041766,-0.101905,-0.142334,0.007385,-0.010490,subj05,0.0005,30000,V1,nsdsyn,0.595493,NSD V1,Extension,NaN
15,2.102069,0.172470,0.213249,0.114772,0.014219,0.129493,-0.170258,0.020818,-0.030411,subj06,0.0005,30000,V1,nsdsyn,1.053529,NSD V1,Extension,NaN
18,2.290964,0.104453,0.098955,0.056838,-0.155388,0.157476,-0.195395,-0.018938,0.002934,subj07,0.0005,30000,V1,nsdsyn,0.512969,NSD V1,Extension,NaN
21,2.554011,0.208712,0.096840,0.053212,-0.016782,-0.317137,-0.148563,0.037118,-0.018485,subj08,0.0005,30000,V1,nsdsyn,0.421552,NSD V1,Extension,NaN
0,2.984467,0.071135,0.399834,-0.003961,-0.020345,0.011555,0.004440,0.011346,-0.005278,wlsubj001,0.0005,30000,V1,broderick,10.369273,Broderick et al. V1,Replication,corrected
1,2.210294,0.192392,0.318329,0.012519,-0.027342,0.038427,-0.009277,0.003553,0.011274,wlsubj006,0.0005,30000,V1,broderick,10.957087,Broderick et al. V1,Replication,corrected


In [17]:
utils.calculate_weighted_mean(final_df, ['slope','intercept'], 'precision', groupby=['dset_type'])


,dset_type,slope,intercept
0,Broderick et al. V1,0.115890,0.369917
1,NSD V1,0.143683,0.150899
2,NSD V2,0.188906,0.199878
3,NSD V3,0.175543,0.155972


# Plot parameter estimates